# Complete code for creation of catalogue

Import libraries

In [ ]:
from astroquery.gaia import Gaia
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from tqdm import tqdm
import os
import shutil
import warnings
from sklearn.neighbors import KernelDensity
from astropy.coordinates import SkyCoord
import astropy.units as u

warnings.filterwarnings("ignore")

Enter to Gaia portal

In [ ]:
user_name = 'slastra'
pass_gaia = '##########'

Gaia.login(user = user_name, password= pass_gaia) # Also: Gaia.login()

path = "/home/santiago/Documents/Tesis/codeComplete"

#Threshold to remove outliers from very small or very large clusters.

lim1_cmd=30
lim2_cmd=900

## Functions necessary 

"$\textit{GAIA_search}$" search all the stars locate it 1 degree around the center of the cluster that have $ruwe <1.4$, $parallax\_over\_error >8$, $parallax>0$ and $phot\_g\_mean\_mag<20$

In [ ]:
def GAIA_search(clust):
    radius  = 1    # Degrees
    inp_ra  = hunt[hunt["name"]==clust]["ra"].values[0]  # Degrees
    inp_dec = hunt[hunt["name"]==clust]["dec"].values[0]  # Degrees

    query = f"SELECT * FROM gaiaedr3.gaia_source JOIN external.gaiaedr3_distance AS dist USING (source_id)\
    WHERE 1=CONTAINS(POINT({inp_ra}, {inp_dec}),CIRCLE(ra, dec, {radius})) AND \
    ruwe <1.4 AND parallax_over_error >8 AND parallax>0 AND phot_g_mean_mag<20"

    job     = Gaia.launch_job_async(query)
    results = job.get_results().to_pandas()

    #Delete NANs
    results = results.dropna(subset = ['parallax']).reset_index()

    data = pd.concat([results['l'],results['b'],results['parallax'],results['pmra'],results['pmdec']],axis=1)
    
    return data,results

"$\textit{best_DBSCAN}$" is used to identify the largest star cluster within that region of space. A broad range of epsilon values must be tested to determine the largest epsilon that results in identifying a single cluster.

In [ ]:
def best_DBSCAN(data,results):
    can_data = 100

    epsilon = np.linspace(0.4,0.03,can_data)

    cumulos = []
    db_value = 0

    for j in range(len(epsilon)): #Cada valor de Min_Samples
        db = DBSCAN(eps=epsilon[j], min_samples=8).fit(data)
        labels = db.labels_

        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        cumulos.append(n_clusters_)  #Cantidad de cúmulos
            
        if n_clusters_ == 1:
            break
    
    ### Se crea el array de estrellas ###
    cluster = data[~labels.astype(bool)]
    cluster_cmd = results.iloc[cluster.index.to_numpy()]
    cluster_cmd = cluster_cmd.dropna(subset=["phot_bp_mean_mag","phot_rp_mean_mag","phot_g_mean_mag"])
    cluster_cmd.drop(["index"], axis = 1, inplace = True)
    
    return cluster_cmd

After applying $\texttt{DBSCAN}$ and $\texttt{pyUPMASK}$ to minimize the outliers in the clusters, it is necessary to run a program called "$\texttt{fitCMD}$," developed in Fortran, to calculate the cluster's isochrone. To properly prepare the data for this process, it must be organized using "$\textit{fitCMD_data}$".

In [ ]:
def fitCMD_data(cluster_pyUPMASK):
    error_g_mean_mag = 2.5/np.log(10)*cluster_pyUPMASK["phot_g_mean_flux_error"]/cluster_pyUPMASK["phot_g_mean_flux"]

    error_rp_mean_mag = 2.5/np.log(10)*cluster_pyUPMASK["phot_rp_mean_flux_error"]/cluster_pyUPMASK["phot_rp_mean_flux"]
    error_bp_mean_mag = 2.5/np.log(10)*cluster_pyUPMASK["phot_bp_mean_flux_error"]/cluster_pyUPMASK["phot_bp_mean_flux"]

    error_bp_rp_color = error_bp_mean_mag + error_rp_mean_mag

    error_g_mean_mag = pd.DataFrame(np.transpose(error_g_mean_mag), columns=["error_g_mean_mag"])
    error_rp_mean_mag = pd.DataFrame(np.transpose(error_rp_mean_mag), columns=["error_rp_mean_mag"])
    error_bp_mean_mag = pd.DataFrame(np.transpose(error_bp_mean_mag), columns=["error_bp_mean_mag"])
    error_bp_rp_color = pd.DataFrame(np.transpose(error_bp_rp_color), columns=["error_bp_rp_color"])

    cluster_ASteCA = pd.concat([cluster_pyUPMASK,error_g_mean_mag,error_bp_rp_color,error_bp_mean_mag,
                                error_rp_mean_mag],axis=1)

    cluster_fitCMD= pd.concat([pd.DataFrame(np.arange(1000,1000+len(cluster_pyUPMASK),1), columns=["solution_id"]),
                cluster_ASteCA['ra'],cluster_ASteCA['dec'],
                cluster_ASteCA['parallax'],cluster_ASteCA['parallax_error'],cluster_ASteCA['pmra'],
                cluster_ASteCA['pmra_error'],cluster_ASteCA['pmdec'],cluster_ASteCA['pmdec_error'],
                cluster_ASteCA['phot_g_mean_mag'],
                cluster_ASteCA['error_g_mean_mag'],
                cluster_ASteCA['phot_bp_mean_mag'],
                cluster_ASteCA['error_bp_mean_mag'],
                cluster_ASteCA['phot_rp_mean_mag'],
                cluster_ASteCA['error_rp_mean_mag']],
              axis=1)
    
    cluster_fitCMD.rename(columns = {'solution_id':'#_r', 'ra':'_RAJ2000','dec':'_DEJ2000', 'parallax':"Plx", 
                                    'parallax_error':'e_Plx', 'pmra':'pmRA', 'pmra_error':'e_pmRA','pmdec':'pmDE',
                                     'pmdec_error':'e_pmDE', 'phot_g_mean_mag':'Gmag','error_g_mean_mag':'e_Gmag',
                                     'phot_bp_mean_mag':'BPmag','error_bp_mean_mag':'e_BPmag', 
                                     'phot_rp_mean_mag':'RPmag', 'error_rp_mean_mag':'e_RPmag'}, inplace = True)
    
    return cluster_fitCMD

# Code to extract the stars from the clusters

The Hunt $\&$ Refert 2023 catalog is used as the reference catalog. This catalog is essential for identifying clusters located around the positions of those listed within it.

In [ ]:
hunt = pd.read_csv('Archives/Hunt-Refert2023.csv',sep=",")
repeated_hunt = pd.DataFrame()
hunt_clean = hunt.copy()

i = np.arange(len(hunt))

### Erase duplicate data (clusters that are 1/3600 degrees near to other)

while len(i) != 0:
    ax = hunt_clean[np.sqrt((hunt_clean.loc[i[0]]["ra"]-hunt_clean["ra"])**2+
                          (hunt_clean.loc[i[0]]["dec"]-hunt_clean["dec"])**2)<=1/3600]
    
    i = np.setdiff1d(i,ax.index.values)
    if len(ax) > 1:
        repeated_hunt = pd.concat([ax ,repeated_hunt],axis=0,ignore_index=True)
        hunt_clean.drop(labels=ax.index.values[1:],axis=0,inplace=True)

hunt_important = hunt_clean[(hunt_clean["n_stars"]<lim2) & (hunt_clean["n_stars"]>lim1)]

After importing the Hunt $\&$ Refert data and removing duplicates, $\textit{GAIA_search}$ and $\textit{best_DBSCAN}$ are used to identify the clusters in that region using the methodology established in this project.

Additionally, $\texttt{pyUPMASK}$ is executed to calculate the membership probability for each star in the cluster.

In [ ]:
for i in tqdm(range(len(hunt_important))):
    clust = hunt_important.iloc[i]["name"]
    data,results = GAIA_search(clust)
    
    cluster_cmd = best_DBSCAN(data,results)
    print("Tamaño del cluster: ", len(cluster_cmd))
    
    if len(cluster_cmd)>lim1_cmd and len(cluster_cmd)<lim2_cmd:
        cluster_cmd.to_csv(path+'/pyUPMASK/input/'+clust+'.csv', index=False)
        results.to_csv(path+'/pyUPMASK/results/'+clust+'.csv', index=False)
        
os.chdir(path+"/pyUPMASK")
os.system("python pyUPMASK.py > pyUPMASK.txt")
os.system("rm pyUPMASK.txt")

Finally, the clusters are processed and prepared for $\texttt{fitCMD}$ using "$\textit{fitCMD_data}$." It is necessary to create a separate folder for each cluster, as $\texttt{fitCMD}$ generates multiple output files. Without proper organization, managing these files can become challenging.

In [ ]:
os.chdir(path+"/pyUPMASK/")
Data=os.listdir("output")

os.chdir(path)
archive= open("names.txt","w+")

print("\n \nFITCMD\n \n")

for i in tqdm(range(len(Data))):
    clust = Data[i][:-4]
    
    cluster_pyUPMASK = pd.read_csv(path+"/pyUPMASK/output/"+clust+".csv",sep=" ")    
    cluster_pyUPMASK = cluster_pyUPMASK[cluster_pyUPMASK["probs_final"]>0.5].reset_index(drop=False)
    cluster_fitCMD = fitCMD_data(cluster_pyUPMASK)
    
    if len(cluster_pyUPMASK)>lim1_cmd and len(cluster_pyUPMASK)<lim2_cmd:
        if i<10:
            cluster_fitCMD.to_csv(path+'/FitCMD/Data/SL40'+str(i)+'_GAIA.DAT', index=False, sep='\t',header=True)
            archive.write("\ncluster:"+clust+" -- Guardado: SL40"+str(i))
        else:
            cluster_fitCMD.to_csv(path+'/FitCMD/Data/SL4'+str(i)+'_GAIA.DAT', index=False, sep='\t',header=True)
            archive.write("\ncluster:"+clust+" -- Guardado: SL4"+str(i))
os.chdir(path)

# Code to create the catalogue

First, the dataframe of the complete catalogue is created with the name and astrophysical data obtained by fitCMD.

The following steps are dedicated to cleaning the data and selecting the clusters with the relevant columns. Additionally, since fitCMD exports the age of clusters in either Myr or Gyr, it is important to standardize the age column by converting all values to the same unit. The same with distance, kpc and pc.

In [ ]:
Data = os.listdir("/home/santiago/Documents/Tesis/TesisComparacion/output")

df_SLR=pd.DataFrame(columns=['name','index','Mcl','DM','E','Age','Z','(m-M)o','dSun','AV','E(B-V)','Z/Zsun'])

for name in Data:
    if name == "S1161":
        continue
    df = pd.read_csv("/home/santiago/Documents/Tesis/TesisComparacion/output/"+name+"/"+name,skiprows=[i for i in range(24)]+[29],
                skipfooter=32,engine='python',sep=":",skipinitialspace = True,names=["Parameter","BestComplete"])

    df["Parameter"]=df["Parameter"].str.replace(' ', '')
    df["BestComplete"]=df["BestComplete"].str.replace('       ', '_')
    df["BestComplete"]=df["BestComplete"].str.replace('      ', '_')
    df["BestComplete"]=df["BestComplete"].str.replace('     ', '_')
    df["BestComplete"]=df["BestComplete"].str.replace('    ', '_')
    df["BestComplete"]=df["BestComplete"].str.replace('   ', '_')
    df["BestComplete"]=df["BestComplete"].str.replace('  ', '_')
    df["BestComplete"]=df["BestComplete"].str.replace(' ', '')

    df[['Best', '+', '-', 'Magnitude']] = df.BestComplete.str.split("_", expand = True)
    df = df[['Parameter','Best', '+', '-', 'Magnitude']]
    df = df.replace('', pd.NA)
    df = df.set_index('Parameter')
    df.Best = df.Best.astype(float)
    df["+"] = df["+"].astype(float)
    df["-"]= df["-"].astype(float)
    df = df.T
    df["name"] = [name for i in range(len(df))]
    df.reset_index(inplace=True)
    df = df[['name','index','Mcl','DM','E','Age','Z','(m-M)o','dSun','AV','E(B-V)','Z/Zsun']]
    df_SLR = pd.concat([df_SLR,df], ignore_index=True)
df_SLR.sort_values(by=['name','index'],ignore_index=True,inplace=True)

In [ ]:
df_SLR_Age=df_SLR[(df_SLR['index']=='Best') | (df_SLR['index']=='Magnitude')]
index = df_SLR_Age[df_SLR_Age.Age == '(Gyr)'].name
best = df_SLR_Age[df_SLR_Age["index"]=="Best"]
best.loc[best['name'].isin(index),'Age'] = best[best['name'].isin(index)].Age.map(lambda Age: Age*1000)

df_SLR_Age = best[["name","Mcl","DM","E","Age","Z","(m-M)o","AV","E(B-V)","Z/Zsun"]]


df_SLR_Dist=df_SLR[(df_SLR['index']=='Best') | (df_SLR['index']=='Magnitude')]
index = df_SLR_Dist[df_SLR_Dist.dSun == '(kpc)'].name
best = df_SLR_Dist[df_SLR_Dist["index"]=="Best"]
best.loc[best['name'].isin(index),'dSun'] = best[best['name'].isin(index)].dSun.map(lambda dSun: dSun*1000)

df_SLR_Dist = best[["dSun"]]

df_SLR = df_SLR_Age.join(df_SLR_Dist,how='inner')
df_SLR.reset_index(inplace=True,drop=True)

Another important step is to categorize the clusters as BAD, BADFIT, MEDIUM, or GOOD. This classification depends directly on the CMD. If the CMD is poor, the cluster is labeled as BAD. If the CMD is good but the fitCMD adjustment is poor, it is labeled as BADFIT. If the CMD and the adjustment are both fair, the cluster is categorized as MEDIUM. Finally, when both the CMD and the fit are good, the cluster is labeled as GOOD. This process is made it manually

In [ ]:
namesGOODBAD = pd.read_csv("/home/santiago/codeTesis/codeComplete/FitCMD/badCumules_SLR.txt",sep="-",names=["name","category"])
namesGOODBAD.replace({'GOODWOW':'GOOD'},inplace=True)

df_SLR = pd.merge(df_SLR,namesGOODBAD,on="name",how="outer")
df_SLR = df_SLR.fillna("BAD")

Add names to the clusters

In [ ]:
df_names = pd.read_csv("/home/santiago/Documents/Tesis/TesisComparacion/names.txt", sep=" ",names=["cluster","name","--","Save","nameSLR"])
df_names = df_names[["name","nameSLR"]]

## Functions of astrophysics coords

In [ ]:
# Mode of the data

def modaData(phis):
    phis = np.array(phis)
    phis_ = np.linspace(phis.min(),phis.max(),10000)
    
    kde = KernelDensity(kernel="gaussian", bandwidth=0.02*(len(phis))**(1/5)).fit(phis[:, np.newaxis])
    log_dens = kde.score_samples(phis_[:, np.newaxis])
    dens = np.exp(log_dens)

    return phis_[np.argmax(dens)]


# 50% radius of the data

def radio50(df,ra_centro,dec_centro):
    df['distancia'] = np.sqrt((df['ra'] - ra_centro)**2 + (df['dec'] - dec_centro)**2)
    df = df.sort_values('distancia')
    indice_50_por_ciento = round(0.5 * len(df))
    radio_50_por_ciento = df.iloc[indice_50_por_ciento]['distancia']
    return radio_50_por_ciento

# Radius for half of the luminosity

def radio50_lum(df,ra_centro,dec_centro):
    df=df.copy()
    df['distancia'] = np.sqrt((df['ra'] - ra_centro)**2 + (df['dec'] - dec_centro)**2) #Distancie to the center
    df = df.sort_values('distancia').reset_index(drop=True) #They are sorted from nearest to furthest.
    half_light = sum(df['phot_g_mean_mag'])/2 #The value for half of the luminosity.
    
    df['cumsum'] = df['phot_g_mean_mag'].cumsum() #Once the indices are sorted, a cumulative sum is performed.
    distance = df[df["cumsum"] <= half_light].tail(1)["distancia"].values[0] #When the sum reaches half_light.
    
    return distance

# Error to the right of the data (84%)

def error_84(array):
    media = array.mean()
    maximo = array.max()
    
    array_84 = np.linspace(media,maximo,1000)
    
    for value in array_84:
        quantity = len(array[array <= value])/len(array)
        if quantity >= 0.84:
            return value
            
# Error to the left of the data (16%)

def error_16(array):
    media = array.mean()
    minimo = array.min()
    
    array_16 = np.linspace(minimo,media,1000)
    
    for value in array_16:
        quantity = len(array[array <= value])/len(array)
        if quantity >= 0.16:
            return value

Creation of the complete catalogue with all the astrophyscis coords and errors

In [ ]:
names = pd.unique(StarsGoodMedium.nameSLR)
coordinates = pd.DataFrame()

for name in tqdm(names):
    df= pd.DataFrame()
    df["name"] = [name]
    df["nStars"] = [len(StarsGoodMedium[StarsGoodMedium.nameSLR == name])]
    
    df["ra"] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].ra)]
    df["ra_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].ra)]
    df["ra_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].ra)]
    
    df['dec'] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].dec)]
    df["dec_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].dec)]
    df["dec_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].dec)]
    
    df['parallax'] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].parallax)]
    df["parallax_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].parallax)]
    df["parallax_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].parallax)]
    
    df['pmra'] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].pmra)]
    df["pmra_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].pmra)]
    df["pmra_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].pmra)]    
    
    df['pmdec'] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].pmdec)]
    df["pmdec_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].pmdec)]
    df["pmdec_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].pmdec)]    
    
    df['l'] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].l)]
    df["l_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].l)]
    df["l_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].l)]  
    
    df['b'] = [modaData(StarsGoodMedium[StarsGoodMedium.nameSLR == name].b)]
    df["b_16"] = [error_16(StarsGoodMedium[StarsGoodMedium.nameSLR == name].b)]
    df["b_84"] = [error_84(StarsGoodMedium[StarsGoodMedium.nameSLR == name].b)]   
    
    df['r50'] = [radio50(StarsGoodMedium[StarsGoodMedium.nameSLR == name][["ra","dec"]],
                         df.ra.values[0],df.dec.values[0])]
    
    df['rhl'] = [radio50_lum(StarsGoodMedium[StarsGoodMedium.nameSLR == name],
                             df.ra.values[0],df.dec.values[0])]
    
    coordinates = pd.concat([coordinates,df],ignore_index=True,axis=0)
    
df_SLR = pd.merge(coordinates,df_SLR,how="inner",on='name')

Also add X,Y,Z coords from SkyCoord

In [ ]:
distancia = df_SLR["dSun"].astype(float).values * u.pc  # Por ejemplo, 100 parsecs
longitud = df_SLR["l"] * u.deg   # Longitud galáctica
latitud = df_SLR["b"] * u.deg    # Latitud galáctica

# Crear un objeto SkyCoord con las coordenadas galácticas
coordenadas_galacticas = SkyCoord(l=longitud, b=latitud, distance=distancia, frame='galactic')

# Obtener las coordenadas cartesianas x, y, z
df_SLR["X"] = pd.DataFrame(coordenadas_galacticas.cartesian.x, columns=["X"])
df_SLR["Y"] = pd.DataFrame(coordenadas_galacticas.cartesian.y, columns=["Y"])
df_SLR["Z"] = pd.DataFrame(coordenadas_galacticas.cartesian.z, columns=["Z"])

## Additionally, create a DataFrame containing all the stars within the clusters

All the stars are imported

In [ ]:
Stars_Data = os.listdir("/home/santiago/codeTesis/codeComplete/pyUPMASK/RunsCode/Total_output/")

Stars=pd.DataFrame()

for name in Stars_Data:
    df = pd.read_csv("/home/santiago/codeTesis/codeComplete/pyUPMASK/RunsCode/Total_output/"+name, sep=" ",header=0)
    df["name"] = [name[:-4]]*len(df)
    Stars = pd.concat([Stars,df],ignore_index=True,axis=0)

The categories of the clusters are analyzed.

In [ ]:
goodMedium = pd.merge(namesGOODBAD,df_names,how="left",left_on='name', right_on='nameSLR')
goodMedium = goodMedium[["name_x","name_y","category"]]
goodMedium = goodMedium.rename(columns={"name_x":"nameSLR","name_y":"name"})
goodMedium = goodMedium[(goodMedium.category == "GOOD") | (goodMedium.category == "MEDIUM")]

Stars classified as GOOD or MEDIUM are selected.

In [ ]:
StarsGoodMedium = Stars[Stars.name.isin(goodMedium.name)]

A join is performed with nameSLR.

In [ ]:
StarsGoodMedium = pd.merge(StarsGoodMedium,df_names,how="left")

### Export catalogue and stars from catalogue

In [ ]:
df_SLR.to_csv("SLR_catalogue_570.csv",sep=";",index=False)
StarsGoodMedium.to_csv("SLR_members.csv",sep=";",index=False)